In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

In [ ]:
from tqdm import tqdm_notebook
from PIL import Image, ImageFilter, ImageChops, ImageOps

In [ ]:
!rm -r ../train
!rm -r ../test

In [ ]:
!mkdir ../train
!mkdir ../test
!mkdir ../train/0
!mkdir ../train/1


!mkdir ../test/0
!mkdir ../test/1

In [ ]:
data = pd.read_csv('/kaggle/input/train_labels.csv')

In [ ]:
data.head()

In [ ]:
one = data.query("label == 1")['id'].values
one_train = one[:len(one) * 4 //5]
one_test = one[len(one) * 4 //5:]

zero = data.query("label == 0")['id'].values
zero_train = zero[:len(zero) * 4 //5]
zero_test = zero[len(zero) * 4 //5:]

In [ ]:
ORIGINAL_SIZE = 96
CROP_SIZE = 32

def process(path, mode, label):

    '''
    colorImage = Image.open("../input/train/%s.tif" % (path))
    
    
    start_crop = (ORIGINAL_SIZE - CROP_SIZE) // 2
    end_crop = start_crop + CROP_SIZE
    colorImage = colorImage.crop((start_crop, start_crop, end_crop, end_crop))

    
    for I in range(4):
        colorImage = colorImage.transpose(Image.ROTATE_90)
        colorImage.save("../%s/%d/%s_%d.png" % (mode, label, path, I), "PNG")
        
    colorImage = colorImage.transpose(Image.FLIP_LEFT_RIGHT)

    for I in range(4):
        colorImage = colorImage.transpose(Image.ROTATE_90)
        colorImage.save("../%s/%d/%s_%d.png" % (mode, label, path, I+4), "PNG")
    
    colorImage = colorImage.transpose(Image.FLIP_TOP_BOTTOM)
    
    for I in range(4):
        colorImage = colorImage.transpose(Image.ROTATE_90)
        colorImage.save("../%s/%d/%s_%d.png" % (mode, label, path, I+8), "PNG")  

    colorImage = colorImage.transpose(Image.FLIP_LEFT_RIGHT)
    
    for I in range(4):
        colorImage = colorImage.transpose(Image.ROTATE_90)
        colorImage.save("../%s/%d/%s_%d.png" % (mode, label, path, I+12), "PNG")  
    
    
    '''    
    colorImage = Image.open("../input/train/%s.tif" % (path))
    start_crop = (ORIGINAL_SIZE - CROP_SIZE) // 2
    end_crop = start_crop + CROP_SIZE
    colorImage = colorImage.crop((start_crop, start_crop, end_crop, end_crop))

    colorImage.save("../%s/%d/%s.png" % (mode, label, path), "PNG")
    

In [ ]:
def myFunc(image):
    image = np.array(image)
    return image / 255

In [ ]:
for I in tqdm_notebook(one_train):
    process(I, "train", 1)
for I in tqdm_notebook(one_test):
    process(I, "test", 1)
for I in tqdm_notebook(zero_train):
    process(I, "train", 0)
for I in tqdm_notebook(zero_test):
    process(I, "test", 0)

In [ ]:
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.models import Sequential
from keras import backend as K
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, warnings
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Dense
from keras.models import Model
from keras.engine.topology import get_source_inputs
from keras.utils import get_file
from keras.utils import layer_utils


sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

In [ ]:
def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x


# Original SqueezeNet from paper.

def SqueezeNet():


    inputs = Input(shape=(32,32,3))


    x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(inputs)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    x = GlobalAveragePooling2D()(x)
    x = Dense(1, activation='sigmoid')(x)

    
    model = Model(inputs, x, name='squeezenet')

    return model

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator()
train_it = datagen.flow_from_directory('../train/', class_mode='binary', batch_size=128, target_size=(32,32))

In [ ]:
test_it = datagen.flow_from_directory('../test/', class_mode='binary', batch_size=128, target_size=(32,32))

In [ ]:
model = SqueezeNet()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_it, steps_per_epoch=128, epochs=5, validation_data=test_it, validation_steps=128)